In [1]:
# Genome Reconstruction by k-mers - De Druijn Graph Approach 

In [47]:
# Please enter the file names of reads and their corresponding output files here:
'''
read_pairs = 'read1.txt'
output_file_read = 'read1_out.txt'
out1 = 'TAATGCCATGGGATGTT'
kmer_read_pairs = 3
read_k= 3
read_d = 4
'''


read_pairs = 'read2.txt'
output_file_read = 'read2_out.txt'
out2 = 'GTGGTCGTGAGATGTTGA'
kmer = 4
read_k= 4
read_d = 2


'''

read_pairs = 'read3.txt'
output_file_read = 'read3_out.txt'

kmer_read_pairs = 30
read_k= 200
read_d = 50

'''



"\n\nread_pairs = 'read3.txt'\noutput_file_read = 'read3_out.txt'\n\nkmer_read_pairs = 30\nread_k= 200\nread_d = 50\n\n"

In [48]:
import numpy as np
import random

In [49]:
#Creates kmers of k size for sequence couples. Returns a list of tuples consisting of kmer couples.

def build_kmers1(seq_couple, k):
    seq0 = seq_couple[0]
    seq1 = seq_couple[1]
    l = [(seq0[i:k+i],seq1[i:k+i]) for i in range(0,len(seq0)-k+1)]
    
    return l

In [50]:
#Forms the debruijn graph from the given reads

def debruijnize(debruijn_map, reads, possible_starts, not_starts):
    edges = []
    nodes = set()
    for r in reads:
        prefix0 = r[0][:-1]
        suffix0 = r[0][1:]
        prefix1 = r[1][:-1]
        suffix1 = r[1][1:]
        edge = ((prefix0, prefix1), (suffix0, suffix1))
        
        possible_starts.add(edge[0])
        not_starts.add(edge[1])
        
        edges.append(edge)
        nodes.add((prefix0,prefix1))
        
    for edge in edges: 
        pre = edge[0]
        suf = edge[1]
        if pre not in debruijn_map:
            debruijn_map[pre] = [suf]
        else:
            debruijn_map[pre].append(suf)
            
    return debruijn_map, nodes

In [51]:
#Finds the eulerian path from the given debruijn graph

def eulerian_finder(graph, start):
    
    insert_point = 0
    first_start = start
    current_n = start
    all_visited = []
    
    while len(graph) != 0:
        found_new_start = False
        current_n = start
        visited = [current_n]
        #print('current n', current_n)
        while len(graph) != 0:
            if current_n not in graph.keys(): 
                break
            next_n = graph[current_n].pop(0)
            visited.append(next_n)
            if len(graph[current_n]) == 0:
                graph.pop(current_n, None)
            else:
                #If the node is not empty, it is stored as the start point for the next round
                start = current_n
                insert_point = len(all_visited)
                found_new_start = True
            current_n = next_n
            
        #Insert the newly found cycle to the path
        all_visited = all_visited[: insert_point-1] + visited + all_visited[insert_point-1 :] 
        if len(graph) <= 0:
            break
        if not found_new_start:
            break

    return all_visited

In [52]:
# For the adjacency table

import pandas as pd 
from IPython.core.display import HTML,display

In [53]:
# Builds the adjacency table for the given nodes from the whole debruijn graph. Returns HTML table

def build_adjacency(nodes, graph): 
    edges = []
    for pre in nodes:
        sufs = graph[pre]
        for s in sufs:
            edges.append((pre, s))

    df = pd.DataFrame(edges)
    adj_matrix = pd.crosstab(df[0], df[1])

    table = adj_matrix.to_html()

    return HTML(table)

In [63]:
# Runs the kmer - debruijn graph algorithm for a txt file of reads.

def file_handler(file, kmer):
    
    raw_read_pairs = []
    read_start = None
    with open(file) as f:
        for line in f:
            reads = line.strip().split('|')
            raw_read_pairs.append(reads)
    debruijn_graph = {}
    reads = []
    nodes_for_adj = set()

    possible_starts = set()
    not_starts = set()

    for r in raw_read_pairs:
        read = build_kmers1(r,kmer)
        reads.append(read)
        m, n = debruijnize(debruijn_graph, read, possible_starts, not_starts)
        if len(nodes_for_adj) <= 20: 
           #print(len(nodes))
            nodes_for_adj = set.union(nodes_for_adj, n)
    starts = list(possible_starts-not_starts)

    if len(starts)!= 1:
        print('Data set has more than one starting node, please recheck the data')
        print('Number of fake starting nodes', len(starts))
        
        return debruijn_graph, None, nodes_for_adj
    else:
        read_start = starts[0]
        
        return debruijn_graph, read_start, nodes_for_adj

In [64]:
# Transforms the final eulerian path into the final read

def read_transformer(euler_path, distance):
    read = euler_path[0][0]
    read2 = euler_path[0][1] 
    c = 0
    for node in euler_path:
        if c > 0:
            read += node[0][-1]
            read2 += node[1][-1]
        c+=1
    print(read)
    print(read2)
    final_read = read + read2[-distance:]
    
    return final_read

In [65]:
debruijn_graph, start_node, nodes_for_adj = file_handler(read_pairs, kmer_read_pairs)

#Builds adjacency table
adj_matrix = build_adjacency(nodes_for_adj,debruijn_graph)
display(adj_matrix)

#Builds the eulerian path and corresponding read
if start_node != None:
    path = eulerian_finder(debruijn_graph, start_node)
    output = read_transformer(path, (read_k+ read_d))

1,"(AG, TG)","(CG, AT)","(GA, GA)","(GA, TT)","(GG, GA)","(GT, AG)","(GT, TG)","(TC, GA)","(TG, GT)","(TG, TG)"
0,,,,,,,,,,
"(AG, TG)",0,0,1,0,0,0,0,0,0,0
"(CG, AT)",0,0,0,0,0,0,2,0,0,0
"(GA, TT)",2,0,0,0,0,0,0,0,0,0
"(GG, GA)",0,0,0,0,0,2,0,0,0,0
"(GT, AG)",0,0,0,0,0,0,0,2,0,0
"(GT, GT)",0,0,0,0,0,0,0,0,0,1
"(GT, TG)",0,0,0,0,0,0,0,0,2,0
"(TC, GA)",0,2,0,0,0,0,0,0,0,0
"(TG, GT)",0,0,0,2,0,0,0,0,0,0


GTGGTCGTGAGAGA
GTGAGATGTTGTGA


In [68]:
print(output)
'GTGGTCGTGAGATGTTGA'

GTGGTCGTGAGAGATTGTGA


'GTGGTCGTGAGATGTTGA'

In [69]:
import Bio
from Bio import pairwise2

In [70]:
with open(output_file_read, 'r') as f:
    out_rosalind_long = f.read()
alignments = pairwise2.align.globalxx(output, out_rosalind_long)
print(alignments[0])

Alignment(seqA='GTGGTCGTGAGAGATTGT-GA', seqB='GTGGTCGT--GAGA-TGTTGA', score=17.0, start=0, end=21)
